## PART B

### Q3. Sentiment Analysis 
In this section, the sentiment of the reviews has been looked at. To build the sentiment classifer, the deception classifer has been retained although modified.The review rating has been set as the sentiment gold standard. Reviews that got 1-2 stars are classified as negative reviews and the reviews that got 4-5 stars are positive. The rating has been used as the feature label. Previously in the Deception Classifier, the real or fake labels have been used. Whereas here, positive or negative rating has been set as the label. The rating is the 3rd column in the dataset. That is line[2] when setting it in the defined python function ParseReview. Accordingly, the load data function has been modified to append the new positive and negative lists. neg_data and pos_data have been initialised as empty lists when loading the dataset and calling the loadData function. Another aspect that was considered is the size of the positive and negative reviews. There are 16183 positive reviews whereas only 2949 negative reviews. As such, when testing the data, before splitting the data into train and test, the rawData function has been set to only take 2500 each of positive and negative reviews so that there is a balanced dataset. It can be seen that for the sentiment classifer the accuracy improves, using only the standard features that is only ID and the review text. The accuracy scores have been reported in the report. Also, additional features have been added to the default features of the Review Text and ID. Each additional feature has been replaced in the LoadData and ParseReview function. The features have been updated to the dictionary in the splitData function. For example first product title was added, and then replaced by verified purchase in the LoadData, ParseReview and SplitData function.

In [1]:
import sys
import nltk
# coding: utf-8
import csv  
import unicodecsv   # csv reader
import numpy as np
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline
import re
from sklearn.metrics import precision_recall_fscore_support # to report on precision and recall
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from nltk.corpus import stopwords
from string import punctuation
from sklearn.metrics import *

In [2]:
# load data from a file and append it to the rawData
def loadData(path, Text=None):
    with open(path, 'rb') as f:
        reader = unicodecsv.reader(f, encoding='utf-8', delimiter='\t')
        next(reader)
        for line in reader:
            (Id, Text, Label, Category) = parseReview(line)
            if Label == "negative":
                neg_data.append((Id, Text, Label, Category))
            elif Label == "positive":
                pos_data.append((Id, Text, Label, Category))
#             rawData is now a combination of pos_data and neg_data›
#             rawData.append((Id, Text, Label))
            preprocessedData.append((Id, preProcess(Text), Label, Category))
            

In [3]:
def splitData(percentage):
    dataSamples = len(rawData)
    halfOfData = int(len(rawData)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    for (_, Text, Label, Category) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
        d = (toFeatureVector(preProcess(Text)))
        d.update({"Product_Category": Category})
        trainData.append((d,Label))
#         trainData.append((toFeatureVector(preProcess(Text)),Label))
    for (_, Text, Label, Category) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text)),Label))

In [4]:
# Positive and negative reviews
Pos = 0
Neg = 0
three = 0
# N, P, T = 0, 0, 0

# DOC_ID	LABEL	RATING	VERIFIED_PURCHASE	PRODUCT_CATEGORY	PRODUCT_ID	PRODUCT_TITLE	REVIEW_TITLE	REVIEW_TE
# Convert line from input file into an id/text/label tuple
def parseReview(reviewLine):
    # Use rating as a label and selecting positive and negative reviews
    Id = reviewLine[0]
    Text = reviewLine[8]
    Prod_Title = reviewLine[6]
    Review_Title = reviewLine[7]
    Verified = reviewLine[3]
    Category = reviewLine[4]
    Label = reviewLine[2]
    if int(reviewLine[2]) < 3:
        Label = "negative"
        global Neg
        Neg += 1
    elif int(reviewLine[2]) >= 4:
        Label = "positive"
        global Pos
        Pos += 1
    else:
        Label = "none"
        global three
        three += 1


    return (Id, Text, Label, Category)

# TEXT PREPROCESSING

def preProcess(text):
    #normalisation and tokenising 
    no_symbols = re.sub(r'[^\w]', ' ', text.lower())
    tokens = no_symbols.split()
    return tokens

In [5]:
## QUESTION 2 ##
# ################
featureDict = {} # A global dictionary of features

def toFeatureVector(words):
    v = {}
    for w in words:
        try:
            i = featureDict[w]
        except KeyError:
            i = len(featureDict) + 1
            featureDict[w] = i
        try:
            v[w] += (1.0/len(words))
        except KeyError:
            v[w] = (1.0/len(words))
    return v



In [9]:
print(Pos)
print(Neg)
print(three)
sum = 16183+2949+1868
print(sum)

16183
2949
1868
21000


In [6]:
################
## QUESTION 3 ##
################
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainData)


myTestData = []
myTrainData = []

def crossValidate(dataset, folds):
    cv_results = []
    accuracy = []
    shuffle(dataset)
    foldSize = int(len(dataset)/folds)
    for i in range(0,len(dataset),foldSize):
        # insert code here that trains and tests on the 10 folds of data in the dataset
        print ("fold start %d foldSize %d" % (i, foldSize))
        myTestData = dataset[i:i+foldSize]
        myTrainData = dataset[0:i] + dataset[i+foldSize:]
        classifier = trainClassifier(myTrainData)
        y_pred = predictLabels(myTestData, classifier)
#         review,label = zip(*myTestData)
#         y_true = label
        y_true = [x[1] for x in myTestData]
#         y_true = classifier.classify(map(lambda x: x[1], myTestData))
        cv_results.append(precision_recall_fscore_support(y_true, y_pred, average='weighted'))
        accuracy.append(accuracy_score(y_true, y_pred))
        
#Calclualte avergae of values over the 10-fold runs
    cv_results = np.asarray(cv_results)
    cv_results = [np.mean(cv_results[:,0]), np.mean(cv_results[:,1]), np.mean(cv_results[:,2])]
    
    accuracy = np.asarray(accuracy)
    accuracy = np.mean(accuracy)
    cv_results.append(accuracy)
    
    return cv_results
# PREDICTING LABELS GIVEN A CLASSIFIER


def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))

def predictLabel(reviewSample, classifier):
    return classifier.classify(toFeatureVector(preProcess(reviewSample)))

In [7]:
# MAIN

# loading reviews
rawData = []          # the filtered data from the dataset file (should be 21000 samples)
preprocessedData = [] # the preprocessed reviews (just to see how your preprocessing is doing)
trainData = []        # the training data as a percentage of the total dataset (currently 80%, or 16800 samples)
testData = []  # the test data as a percentage of the total dataset (currently 20%, or 4200 samples)
pos_data = []
neg_data = []
# the output classes
# fakeLabel = 'fake'
# realLabel = 'real'

# references to the data files
reviewPath = 'amazon_reviews.txt'

## Do the actual stuff
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')
loadData(reviewPath)
# We split the raw dataset into a set of training data and a set of test data (80/20)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')

# selecting rawData to be of a balanced dataset
rawData = pos_data[:2500] + neg_data[:2500]

splitData(0.8)
# We print the number of training samples and the number of features
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')

Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 0 rawData, 0 trainData, 0 testData
Preparing training and test data...
Now 5000 rawData, 4000 trainData, 1000 testData
Training Samples: 
4000
Features: 
14888


In [8]:
crossValidate(trainData, 10)

fold start 0 foldSize 400
Training Classifier...
fold start 400 foldSize 400
Training Classifier...
fold start 800 foldSize 400
Training Classifier...
fold start 1200 foldSize 400
Training Classifier...
fold start 1600 foldSize 400
Training Classifier...
fold start 2000 foldSize 400
Training Classifier...
fold start 2400 foldSize 400
Training Classifier...
fold start 2800 foldSize 400
Training Classifier...
fold start 3200 foldSize 400
Training Classifier...
fold start 3600 foldSize 400
Training Classifier...


[0.7789407899415657, 0.7780000000000001, 0.777974589526049, 0.778]